<a href="https://colab.research.google.com/github/szhang12345/10-Python-Practice-Questions/blob/master/MSDS_458_Siying_Zhang_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MSDS458 Research Assignment 2
## Siying Zhang

The CIFAR-10 dataset (Canadian Institute For Advanced Research) is a collection of images that are commonly used to train machine learning and computer vision algorithms. It is one of the most widely used datasets for machine learning research. The CIFAR-10 dataset contains 60,000 32x32 color images in 10 different classes. The 10 different classes represent airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. There are 6,000 images of each class.

## Import packages needed 

In [ ]:
# Helper libraries
import datetime
from packaging import version
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from collections import Counter
import numpy as np
import pandas as pd

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Input, Dense

In [ ]:
%matplotlib inline
np.set_printoptions(precision=3, suppress=True)

### Verify TensorFlow Version and Keras Version

In [ ]:
print("This notebook requires TensorFlow 2.0 or above")
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >=2

In [ ]:
print("Keras version: ", keras.__version__)

## Loading cifar10 Dataset

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.<br>

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.


In [ ]:
(train_images, train_labels),(test_images, test_labels)= tf.keras.datasets.cifar10.load_data()

* Tuple of Numpy arrays: (x_train, y_train), (x_test, y_test).
* x_train, x_test: uint8 arrays of color image data with shapes (num_samples, 32, 32).
* y_train, y_test: uint8 arrays of digit labels (integers in range 0-9)

## EDA Training and Test Datasets

* Imported 50000 examples for training and 10000 examples for test 
* Imported 50000 labels for training and 10000 labels for test 

In [ ]:
print('train_images:\t{}'.format(train_images.shape))
print('train_labels:\t{}'.format(train_labels.shape))
print('test_images:\t\t{}'.format(test_images.shape))
print('test_labels:\t\t{}'.format(test_labels.shape))

### Review labels for training dataset

In [ ]:
print("First ten labels training dataset:\n {}\n".format(train_labels[0:10]))
print("This output the numeric label, need to convert to item description")

### Plot Examples

In [ ]:
def get_three_classes(x, y):
    def indices_of(class_id):
        indices, _ = np.where(y == float(class_id))
        return indices

    indices = np.concatenate([indices_of(0), indices_of(1), indices_of(2)], axis=0)
    
    x = x[indices]
    y = y[indices]
    
    count = x.shape[0]
    indices = np.random.choice(range(count), count, replace=False)
    
    x = x[indices]
    y = y[indices]
    
    y = tf.keras.utils.to_categorical(y)
    
    return x, y

In [ ]:
x_preview, y_preview = get_three_classes(train_images, train_labels)
x_preview_test, y_preview_test = get_three_classes(test_images, test_labels)

In [ ]:
class_names_preview = ['aeroplane', 'car', 'bird']

def show_random_examples(x, y, p):
    indices = np.random.choice(range(x.shape[0]), 10, replace=False)
    
    x = x[indices]
    y = y[indices]
    p = p[indices]
    
    plt.figure(figsize=(10, 5))
    for i in range(10):
        plt.subplot(2, 5, i + 1)
        plt.imshow(x[i])
        plt.xticks([])
        plt.yticks([])
        col = 'green' if np.argmax(y[i]) == np.argmax(p[i]) else 'red'
        plt.xlabel(class_names_preview[np.argmax(p[i])], color=col)
    plt.show()

show_random_examples(x_preview, y_preview, y_preview)

### Random Review of Examples 

In [ ]:
show_random_examples(x_preview, y_preview, y_preview)

## Preprocessing Data for Model Development

The labels are an array of integers, ranging from 0 to 9. These correspond to the class of clothing the image represents:

|Label  |Class_  |
|-------|--------|
|0|	airplane     |
|1|	automobile   |
|2|	bird         |
|3|	cat          |
|4|	deer         |
|5|	dog          |
|6|	frog         |
|7|	horse        |
|8|	ship         |
|9|	truck        |

In [ ]:
class_names = ['airplane'
,'automobile'
,'bird'
,'cat'
,'deer'
,'dog'
,'frog' 
,'horse'
,'ship'
,'truck']

### Image Shape

The images are 32x32 NumPy arrays, with pixel values ranging from 0 to 255.

1. Each element in each example is a pixel value
2. Pixel values range from 0 to 255
3. 0 = black
4. 255 = white

### Preprocessing the Examples for DNN

In [ ]:
train_dnn = np.reshape(train_images,(50000,3072))
test_dnn = np.reshape(test_images,(10000,3072))
train_dnn = train_dnn.astype('float32')
test_dnn = test_dnn.astype('float32')

# Normalization of pixel values (to [0-1] range)

train_dnn /= 255
test_dnn /= 255

In [ ]:
train_dnn.shape, test_dnn.shape

### Preprocessing the Examples for CNN
 

In [ ]:
train_cnn = train_images.astype('float32')/255.
test_cnn = test_images.astype('float32')/255.

In [ ]:
train_cnn.shape, test_cnn.shape

## Validating our approaches

3,000 samples of our training data to use as a validation set. 

##### DNN

In [ ]:
val_dnn, train_dnn = train_dnn[:3000], train_dnn[3000:] 
val_labels_dnn, train_labels_dnn = train_labels[:3000], train_labels[3000:]

In [ ]:
val_dnn.shape, val_labels_dnn.shape

In [ ]:
train_dnn.shape, train_labels_dnn.shape

##### CNN

In [ ]:
train_cnn = train_images.astype('float32')/255.
test_cnn = test_images.astype('float32')/255.

In [ ]:
train_cnn.shape, test_cnn.shape

In [ ]:
val_cnn, train_cnn = train_cnn[:3000], train_cnn[3000:] 
val_labels_cnn, train_labels_cnn = train_labels[:3000], train_labels[3000:]

In [ ]:
val_cnn.shape, val_labels_cnn.shape

In [ ]:
train_cnn.shape, train_labels_cnn.shape

## Create the Model

Metrics to track performance

In [ ]:
keras.backend.clear_session()

### DNN Model Topology

### Experiment 1: DNN with 1 layer (no regularization)

In [ ]:
model1 = models.Sequential(name="model1")
model1.add(layers.Dense(128, activation='relu', input_shape=(32 * 32 * 3,)))
model1.add(layers.Dense(10, activation='softmax'))

In [ ]:
keras.utils.plot_model(model1, "CIFAR10_model_1hnode.png", show_shapes=True) # plot a graph of the model

In [ ]:
model1.summary() # prints a summary representation of the odel

#### Compiling Model

In [ ]:
# For use with non-categorical labels
model1.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


#### Training Model

In [ ]:
# Start timer
start = datetime.datetime.now()
history = model1.fit(train_dnn
                    ,train_labels_dnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_dnn,val_labels_dnn)
                    ,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)])
# Record the time it takes
duration = datetime.datetime.now() - start

#### Model Predictions

In [ ]:
test_loss, test_accuracy = model1.evaluate(test_dnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model1.predict(test_dnn)
print('shape of preds: ', preds.shape)

#### Model Performance and Evaluation

In [ ]:
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
# Initialize a dictionary to track results
metrics = list()
data=dict()

# Metrics for evaluation
names = ['Model Name', 'Training Accuracy', 
         'Validation Accuracy', 'Test Accuracy','Training Loss', 
         'Validation Loss', 'Test Loss'
         'Time (s)']


In [ ]:
metrics = list()
data=dict()
data['Model Name'] = "Model1 DNN with 1 layer (no regularization)"
data['train_loss'] = history_dict['loss'][-1]
data['train_acc'] = history_dict['accuracy'][-1]
data['val_loss'] = history_dict['val_loss'][-1]
data['val_acc'] = history_dict['val_accuracy'][-1]
data['Test Accuracy']=test_accuracy
data['Test Loss']=test_loss
data['Time (s)']=duration
metrics.append(data)

In [ ]:
metrics

### Experiment 2: DNN with 2 layers (no regularization)

In [ ]:
model2 = models.Sequential(name="model2")
model2.add(layers.Dense(128, activation='relu', input_shape=(32 * 32 * 3,)))
model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dense(10, activation='softmax'))
model2.summary() # prints a summary representation of the odel


In [ ]:
# For use with non-categorical labels
model2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Start timer
start = datetime.datetime.now()
history = model2.fit(train_dnn
                    ,train_labels_dnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_dnn,val_labels_dnn)
                    ,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)])
# Record the time it takes
duration = datetime.datetime.now() - start

In [ ]:
test_loss, test_accuracy = model2.evaluate(test_dnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model2.predict(test_dnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
metrics2=list()
data2=dict()
data2['Model Name'] = "Model2 DNN with 2 layer (no regularization)"
data2['train_loss'] = history_dict['loss'][-1]
data2['train_acc'] = history_dict['accuracy'][-1]
data2['val_loss'] = history_dict['val_loss'][-1]
data2['val_acc'] = history_dict['val_accuracy'][-1]
data2['Test Accuracy']=test_accuracy
data2['Test Loss']=test_loss
data2['Time (s)']=duration
metrics2.append(data2)
metrics2

### Experiment 3: DNN with 3 layers (no regularization)

In [ ]:
model3 = models.Sequential(name="model3")
model3.add(layers.Dense(128, activation='relu', input_shape=(32 * 32 * 3,)))
model3.add(layers.Dense(256, activation='relu'))
model3.add(layers.Dense(512, activation='relu'))
model3.add(layers.Dense(10, activation='softmax'))
model3.summary() # prints a summary representation of the odel
# For use with non-categorical labels
model3.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Start timer
start = datetime.datetime.now()
history = model3.fit(train_dnn
                    ,train_labels_dnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_dnn,val_labels_dnn)
                    ,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)])
# Record the time it takes
duration = datetime.datetime.now() - start

In [ ]:
test_loss, test_accuracy = model3.evaluate(test_dnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model3.predict(test_dnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
metrics3=list()
data3=dict()
data3['Model Name'] = "Model3 DNN with 3 layer (no regularization)"
data3['train_loss'] = history_dict['loss'][-1]
data3['train_acc'] = history_dict['accuracy'][-1]
data3['val_loss'] = history_dict['val_loss'][-1]
data3['val_acc'] = history_dict['val_accuracy'][-1]
data3['Test Accuracy']=test_accuracy
data3['Test Loss']=test_loss
data3['Time (s)']=duration
metrics3.append(data3)
metrics3

### Experiment 4: DNN with 2 layers (batch normalization regularization)

In [ ]:
from keras.layers import BatchNormalization

In [ ]:
model4 = models.Sequential(name="model4")
model4.add(layers.Dense(128, activation='relu', input_shape=(32 * 32 * 3,)))
model4.add(layers.BatchNormalization())
model4.add(layers.Dense(256, activation='relu'))
model4.add(layers.BatchNormalization())
model4.add(layers.Dense(10, activation='softmax'))
model4.summary() # prints a summary representation of the odel
# For use with non-categorical labels
model4.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Start timer
start = datetime.datetime.now()
history = model4.fit(train_dnn
                    ,train_labels_dnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_dnn,val_labels_dnn)
                    ,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)])
# Record the time it takes
duration = datetime.datetime.now() - start

In [ ]:
test_loss, test_accuracy = model4.evaluate(test_dnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model4.predict(test_dnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
metrics4=list()
data4=dict()
data4['Model Name'] = "Model4 DNN with 2 layer (batch normalization regularization)"
data4['train_loss'] = history_dict['loss'][-1]
data4['train_acc'] = history_dict['accuracy'][-1]
data4['val_loss'] = history_dict['val_loss'][-1]
data4['val_acc'] = history_dict['val_accuracy'][-1]
data4['Test Accuracy']=test_accuracy
data4['Test Loss']=test_loss
data4['Time (s)']=duration
metrics3.append(data4)
metrics3

In [ ]:
metrics3.append(data)

In [ ]:
metrics3.append(data2)

### Experiment 5: DNN with 3 layers (batch normalization regularization)

In [ ]:
model5 = models.Sequential(name="model5")
model5.add(layers.Dense(128, activation='relu', input_shape=(32 * 32 * 3,)))
model5.add(layers.BatchNormalization())
model5.add(layers.Dense(256, activation='relu'))
model5.add(layers.BatchNormalization())
model5.add(layers.Dense(512, activation='relu'))
model5.add(layers.BatchNormalization())
model5.add(layers.Dense(10, activation='softmax'))
model5.summary() # prints a summary representation of the odel
model5.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Start timer
start = datetime.datetime.now()
history = model5.fit(train_dnn
                    ,train_labels_dnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_dnn,val_labels_dnn)
                    ,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)])
# Record the time it takes
duration = datetime.datetime.now() - start

In [ ]:
test_loss, test_accuracy = model5.evaluate(test_dnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model5.predict(test_dnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
metrics5=list()
data5=dict()
data5['Model Name'] = "Model5 DNN with 3 layer (batch normalization regularization)"
data5['train_loss'] = history_dict['loss'][-1]
data5['train_acc'] = history_dict['accuracy'][-1]
data5['val_loss'] = history_dict['val_loss'][-1]
data5['val_acc'] = history_dict['val_accuracy'][-1]
data5['Test Accuracy']=test_accuracy
data5['Test Loss']=test_loss
data5['Time (s)']=duration
metrics3.append(data5)
metrics3

In [ ]:
metrics3=sorted(metrics3, key = lambda i: (i['Model Name']))

### Experiment 6: DNN with 2 layers (dropout regularization-dropout size: 0.2)

In [ ]:
from keras.layers import Dropout

In [ ]:
model6 = models.Sequential(name="model6")
model6.add(layers.Dense(128, activation='relu', input_shape=(32 * 32 * 3,)))
model6.add(Dropout(0.2))
model6.add(layers.Dense(256, activation='relu'))
model6.add(Dropout(0.2))
model6.add(layers.Dense(10, activation='softmax'))
model6.summary() # prints a summary representation of the odel
# For use with non-categorical labels
model6.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Start timer
start = datetime.datetime.now()
history = model6.fit(train_dnn
                    ,train_labels_dnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_dnn,val_labels_dnn)
                    ,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)])
# Record the time it takes
duration = datetime.datetime.now() - start

In [ ]:
test_loss, test_accuracy = model6.evaluate(test_dnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model6.predict(test_dnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
data6=dict()
data6['Model Name'] = "Model6 DNN with 2 layers (dropout regularization-dropout size: 0.2)"
data6['train_loss'] = history_dict['loss'][-1]
data6['train_acc'] = history_dict['accuracy'][-1]
data6['val_loss'] = history_dict['val_loss'][-1]
data6['val_acc'] = history_dict['val_accuracy'][-1]
data6['Test Accuracy']=test_accuracy
data6['Test Loss']=test_loss
data6['Time (s)']=duration
metrics3.append(data6)
metrics3

### Experiment 7: DNN with 3 layers (dropout regularization-0.2)

In [ ]:
model7 = models.Sequential(name="model7")
model7.add(layers.Dense(128, activation='relu', input_shape=(32 * 32 * 3,)))
model7.add(Dropout(0.2))
model7.add(layers.Dense(256, activation='relu'))
model7.add(Dropout(0.2))
model7.add(layers.Dense(512, activation='relu'))
model7.add(Dropout(0.2))
model7.add(layers.Dense(10, activation='softmax'))
model7.summary() # prints a summary representation of the odel
# For use with non-categorical labels
model7.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Start timer
start = datetime.datetime.now()
history = model7.fit(train_dnn
                    ,train_labels_dnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_dnn,val_labels_dnn)
                    ,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)])
# Record the time it takes
duration = datetime.datetime.now() - start

In [ ]:
test_loss, test_accuracy = model7.evaluate(test_dnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model7.predict(test_dnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
data7=dict()
data7['Model Name'] = "Model7 DNN with 3 layers (dropout regularization-0.2)"
data7['train_loss'] = history_dict['loss'][-1]
data7['train_acc'] = history_dict['accuracy'][-1]
data7['val_loss'] = history_dict['val_loss'][-1]
data7['val_acc'] = history_dict['val_accuracy'][-1]
data7['Test Accuracy']=test_accuracy
data7['Test Loss']=test_loss
data7['Time (s)']=duration
metrics3.append(data7)
metrics3


### CNN Model Topology

We use a Sequential class defined in Keras to create our model. The Conv2D and MaxPooling layers handle feature learning.  The last 3 layers, handle classification.  

### Compiling the model

In addition to setting up our model architecture, we also need to define which algorithm should the model use in order to optimize the weights and biases as per the given data. We will use stochastic gradient descent.

We also need to define a loss function. Think of this function as the difference between the predicted outputs and the actual outputs given in the dataset. This loss needs to be minimised in order to have a higher model accuracy. That's what the optimization algorithm essentially does - it minimises the loss during model training. For our multi-class classification problem, categorical cross entropy is commonly used.

Finally, we will use the accuracy during training as a metric to keep track of as the model trains.

### Experiment 8: CNN with 2 convolution/max pooling layers (no regularization)

In [ ]:
model8 = models.Sequential(name="model8")
model8.add(layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu,input_shape=(32, 32, 3)))
model8.add(layers.MaxPool2D((2, 2),strides=2))
model8.add(layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu))
model8.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model8.add(layers.Flatten())
model8.add(layers.Dense(units=384, activation=tf.nn.relu))
model8.add(layers.Dense(units=10, activation=tf.nn.softmax))

In [ ]:
model8.summary()

In [ ]:
model8.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
# Start timer
start = datetime.datetime.now()
history = model8.fit(train_cnn
                    ,train_labels_cnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_cnn,val_labels_cnn)
                    ,callbacks=[
                    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3),
                    tf.keras.callbacks.ModelCheckpoint('./models/model_{val_accuracy:.4f}.h5', save_best_only=True,
                                        save_weights_only=False, monitor='val_accuracy')]                                                                                                           
                   )
#acc_train = accuracy.eval(feed_dict={X: train_images_norm, y: train_labels})
#acc_val = accuracy.eval(feed_dict={X: val_images_norm, y: val_labels})
#loss_train = loss.eval(feed_dict={X: train_images_norm, y: train_labels})
#loss_val = loss.eval(feed_dict={X: val_images_norm, y: val_labels})
#loss, accuracy = model1.evaluate(test_images_norm, test_labels)
#print('test set accuracy: ', accuracy * 100)
#preds = model1.predict(test_images_norm)
#print('shape of preds: ', preds.shape)
# Record the time it takes
duration = datetime.datetime.now() - start


In [ ]:
test_loss, test_accuracy = model8.evaluate(test_cnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model8.predict(test_cnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
metrics3=list(filter(lambda i: i['Model Name'] != 'Model8 CNN with 2 convolution/max pooling layers (no regularization)', metrics3))

In [ ]:
data8=dict()
data8['Model Name'] = "Model8 CNN with 2 convolution/max pooling layers (no regularization)"
data8['train_loss'] = history_dict['loss'][-1]
data8['train_acc'] = history_dict['accuracy'][-1]
data8['val_loss'] = history_dict['val_loss'][-1]
data8['val_acc'] = history_dict['val_accuracy'][-1]
data8['Test Accuracy']=test_accuracy
data8['Test Loss']=test_loss
data8['Time (s)']=duration
metrics3.append(data8)
metrics3


### Experiment 9: CNN with 3 convolution/max pooling layers (no regularization)

In [ ]:
model9 = models.Sequential(name="model9")
model9.add(layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu,input_shape=(32, 32, 3)))
model9.add(layers.MaxPool2D((2, 2),strides=2))
model9.add(layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu))
model9.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model9.add(layers.Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu))
model9.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model9.add(layers.Flatten())
model9.add(layers.Dense(units=384, activation=tf.nn.relu))
model9.add(layers.Dense(units=10, activation=tf.nn.softmax))
model9.summary()
model9.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
# Start timer
start = datetime.datetime.now()
history = model9.fit(train_cnn
                    ,train_labels_cnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_cnn,val_labels_cnn)
                    ,callbacks=[
                    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3),
                    tf.keras.callbacks.ModelCheckpoint('./models/model_{val_accuracy:.4f}.h5', save_best_only=True,
                                        save_weights_only=False, monitor='val_accuracy')]                                                                                                           
                   )

# Record the time it takes
duration = datetime.datetime.now() - start



In [ ]:
test_loss, test_accuracy = model9.evaluate(test_cnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model9.predict(test_cnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
data9=dict()
data9['Model Name'] = "Model9 CNN with 3 convolution/max pooling layers (no regularization)"
data9['train_loss'] = history_dict['loss'][-1]
data9['train_acc'] = history_dict['accuracy'][-1]
data9['val_loss'] = history_dict['val_loss'][-1]
data9['val_acc'] = history_dict['val_accuracy'][-1]
data9['Test Accuracy']=test_accuracy
data9['Test Loss']=test_loss
data9['Time (s)']=duration
metrics3.append(data9)
metrics3


### Experiment 10: CNN with 2 convolution/max pooling layers (batch normalization regularization)

In [ ]:
model10 = models.Sequential(name="model10")
model10.add(layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu,input_shape=(32, 32, 3)))
model10.add(BatchNormalization())
model10.add(layers.MaxPool2D((2, 2),strides=2))
model10.add(layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu))
model10.add(BatchNormalization())
model10.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
#model10.add(layers.Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu))
#model10.add(BatchNormalization())
#model10.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model10.add(layers.Flatten())
model10.add(layers.Dense(units=384, activation=tf.nn.relu))
model10.add(layers.Dense(units=10, activation=tf.nn.softmax))
model10.summary()
model10.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
# Start timer
start = datetime.datetime.now()
history = model10.fit(train_cnn
                    ,train_labels_cnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_cnn,val_labels_cnn)
                    ,callbacks=[
                    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
                    ]                                                                                                           
                   )

# Record the time it takes
duration = datetime.datetime.now() - start


In [ ]:
test_loss, test_accuracy = model10.evaluate(test_cnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model10.predict(test_cnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
data10=dict()
data10['Model Name'] = "Model10 CNN with 2 convolution/max pooling layers (batch normalization regularization)"
data10['train_loss'] = history_dict['loss'][-1]
data10['train_acc'] = history_dict['accuracy'][-1]
data10['val_loss'] = history_dict['val_loss'][-1]
data10['val_acc'] = history_dict['val_accuracy'][-1]
data10['Test Accuracy']=test_accuracy
data10['Test Loss']=test_loss
data10['Time (s)']=duration
metrics3.append(data10)
metrics3


### Experiment 11: CNN with 3 convolution/max pooling layers (batch normalization regularization)

In [ ]:
model11 = models.Sequential(name="model11")
model11.add(layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu,input_shape=(32, 32, 3)))
model11.add(BatchNormalization())
model11.add(layers.MaxPool2D((2, 2),strides=2))
model11.add(layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu))
model11.add(BatchNormalization())
model11.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model11.add(layers.Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu))
model11.add(BatchNormalization())
model11.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model11.add(layers.Flatten())
model11.add(layers.Dense(units=384, activation=tf.nn.relu))
model11.add(layers.Dense(units=10, activation=tf.nn.softmax))
model11.summary()
model11.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
# Start timer
start = datetime.datetime.now()
history = model11.fit(train_cnn
                    ,train_labels_cnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_cnn,val_labels_cnn)
                    ,callbacks=[
                    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
                    ]                                                                                                           
                   )

# Record the time it takes
duration = datetime.datetime.now() - start


In [ ]:
test_loss, test_accuracy = model11.evaluate(test_cnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model11.predict(test_cnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

##### Plotting Performance Metrics 

We use Matplotlib to create 2 plots--displaying the training and validation loss (resp. accuracy) for each (training) epoch side by side.

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
data11=dict()
data11['Model Name'] = "Model11 CNN with 3 convolution/max pooling layers (batch normalization regularization)"
data11['train_loss'] = history_dict['loss'][-1]
data11['train_acc'] = history_dict['accuracy'][-1]
data11['val_loss'] = history_dict['val_loss'][-1]
data11['val_acc'] = history_dict['val_accuracy'][-1]
data11['Test Accuracy']=test_accuracy
data11['Test Loss']=test_loss
data11['Time (s)']=duration
metrics3.append(data11)
metrics3


### Experiment 12: CNN with 2 convolution/max pooling layers (dropout regularization- dropout size: 0.2)

In [ ]:
model12 = models.Sequential(name="model12")
model12.add(layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu,input_shape=(32, 32, 3)))
model12.add(layers.MaxPool2D((2, 2),strides=2))
model12.add(layers.Dropout(0.2))
model12.add(layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu))
model12.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model12.add(layers.Dropout(0.2))
#model10.add(layers.Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu))
#model10.add(BatchNormalization())
#model10.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model12.add(layers.Flatten())
model12.add(layers.Dense(units=384, activation=tf.nn.relu))
model12.add(layers.Dense(units=10, activation=tf.nn.softmax))
model12.summary()
model12.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
# Start timer
start = datetime.datetime.now()
history = model12.fit(train_cnn
                    ,train_labels_cnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_cnn,val_labels_cnn)
                    ,callbacks=[
                    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
                    ]                                                                                                           
                   )

# Record the time it takes
duration = datetime.datetime.now() - start


In [ ]:
test_loss, test_accuracy = model12.evaluate(test_cnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model12.predict(test_cnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

##### Plotting Performance Metrics 

We use Matplotlib to create 2 plots--displaying the training and validation loss (resp. accuracy) for each (training) epoch side by side.

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
data12=dict()
data12['Model Name'] = "Model12 CNN with 2 convolution/max pooling layers (dropout regularization-0.2)"
data12['train_loss'] = history_dict['loss'][-1]
data12['train_acc'] = history_dict['accuracy'][-1]
data12['val_loss'] = history_dict['val_loss'][-1]
data12['val_acc'] = history_dict['val_accuracy'][-1]
data12['Test Accuracy']=test_accuracy
data12['Test Loss']=test_loss
data12['Time (s)']=duration
metrics3.append(data12)
metrics3


### Experiment 13: CNN with 3 convolution/max pooling layers (dropout regularization- dropout size: 0.2)

In [ ]:
model13 = models.Sequential(name="model13")
model13.add(layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu,input_shape=(32, 32, 3)))
model13.add(layers.MaxPool2D((2, 2),strides=2))
model13.add(layers.Dropout(0.2))
model13.add(layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu))
model13.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model13.add(layers.Dropout(0.2))
model13.add(layers.Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), activation=tf.nn.relu))
model13.add(layers.MaxPool2D(pool_size=(2, 2),strides=2))
model13.add(layers.Dropout(0.2))
model13.add(layers.Flatten())
model13.add(layers.Dense(units=384, activation=tf.nn.relu))
model13.add(layers.Dense(units=10, activation=tf.nn.softmax))
model13.summary()
model13.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
# Start timer
start = datetime.datetime.now()
history = model13.fit(train_cnn
                    ,train_labels_cnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_cnn,val_labels_cnn)
                    ,callbacks=[
                    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
                    ]                                                                                                           
                   )

# Record the time it takes
duration = datetime.datetime.now() - start


In [ ]:
test_loss, test_accuracy = model13.evaluate(test_cnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model13.predict(test_cnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

##### Plotting Performance Metrics 

We use Matplotlib to create 2 plots--displaying the training and validation loss (resp. accuracy) for each (training) epoch side by side.

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

In [ ]:
data13=dict()
data13['Model Name'] = "Model13 CNN with 3 convolution/max pooling layers (dropout regularization-0.2)"
data13['train_loss'] = history_dict['loss'][-1]
data13['train_acc'] = history_dict['accuracy'][-1]
data13['val_loss'] = history_dict['val_loss'][-1]
data13['val_acc'] = history_dict['val_accuracy'][-1]
data13['Test Accuracy']=test_accuracy
data13['Test Loss']=test_loss
data13['Time (s)']=duration
metrics3.append(data13)
metrics3


In [ ]:
results=pd.DataFrame(metrics3,columns=['Model Name','Test Accuracy','Test Loss',"Time (s)",'train_acc',"train_loss","val_acc","val_loss"])

In [ ]:
results

In [ ]:
results.set_index('Model Name')

In [ ]:
results.to_csv('results.csv')

In [ ]:
!cp results.csv "drive/My Drive/"

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
pd.set_option('display.max_colwidth', 255)

## Evaluate the model

In order to ensure that this is not a simple "memorization" by the machine, we should evaluate the performance on the test set. This is easy to do, we simply use the `evaluate` method on our model.

## Predictions

In [ ]:
results

In [ ]:
test_loss, test_accuracy = model1.evaluate(test_images_norm, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model1.predict(test_images_norm)
print('shape of preds: ', preds.shape)


## Plotting Performance Metrics 

We use Matplotlib to create 2 plots--displaying the training and validation loss (resp. accuracy) for each (training) epoch side by side.

In [ ]:
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
history_df.tail().round(3)

losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()

## Confusion matrices

Let us see what the confusion matrix looks like. Using both `sklearn.metrics`. Then we visualize the confusion matrix and see what that tells us.

Get the predicted classes

In [ ]:
pred_classes = np.argmax(model13.predict(test_cnn), axis=-1)

### Visualizing the confusion matrix

In [ ]:
conf_mx = tf.math.confusion_matrix(test_labels, pred_classes)
conf_mx

In [ ]:
pd.DataFrame(conf_mx).to_csv('conf.csv')

In [ ]:
!cp conf.csv "drive/My Drive/"

In [ ]:
con_mat = conf_mx.numpy()
classes=[0,1,2,3,4,5,6,7,8,9]
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)

con_mat_df = pd.DataFrame(con_mat_norm,
                     index = classes, 
                     columns = classes)
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues,)
plt.tight_layout()
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.title('CIFAR10 Confusion Matrix Heatmap')
#plt.savefig("confusion_matrix_plot_mnist_1", tight_layout=True)
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
plt.matshow(conf_mx, cmap=plt.cm.Blues,fignum=1)
plt.xlabel("Predicted Classes")
plt.ylabel("Actual Classes")
plt.show()

In [ ]:
preds = model13.predict(test_cnn)

In [ ]:
preds.shape

### Predictions

In [ ]:
cm = sns.light_palette((260, 75, 60), input="husl", as_cmap=True)

In [ ]:
df = pd.DataFrame(preds[0:20], columns = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'])
df.style.format("{:.2%}").background_gradient(cmap=cm)

In [ ]:
(_,_), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

img = test_images[2004]
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)

class_names = ['airplane'
,'automobile'
,'bird'
,'cat'
,'deer'
,'dog'
,'frog' 
,'horse'
,'ship'
,'truck']

plt.imshow(img, cmap='viridis')
plt.axis('off')
plt.show()

In [ ]:
# Extracts the outputs of the top 8 layers:
layer_outputs = [layer.output for layer in model13.layers[:8]]
# Creates a model that will return these outputs, given the model input:
activation_model = models.Model(inputs=model13.input, outputs=layer_outputs)

In [ ]:
activations = activation_model.predict(img_tensor)
len(activations)

In [ ]:
layer_names = []
for layer in model13.layers:
    layer_names.append(layer.name)
    
layer_names

In [ ]:
# These are the names of the layers, so can have them as part of our plot
layer_names = []
for layer in model13.layers[:9]:
    layer_names.append(layer.name)

images_per_row = 16

# Now let's display our feature maps
for layer_name, layer_activation in zip(layer_names, activations):
    # This is the number of features in the feature map
    n_features = layer_activation.shape[-1]

    # The feature map has shape (1, size, size, n_features)
    size = layer_activation.shape[1]

    # We will tile the activation channels in this matrix
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # We'll tile each filter into this big horizontal grid
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            # Post-process the feature to make it visually palatable
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    # Display the grid
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    
plt.show();

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))
# compile model

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
# Start timer
start = datetime.datetime.now()
history = model.fit(train_cnn
                    ,train_labels_cnn
                    ,epochs=200
                    ,batch_size=100
                    ,validation_data=(val_cnn,val_labels_cnn)
                    ,callbacks=[
                    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
                    ]                                                                                                           
                   )

# Record the time it takes
duration = datetime.datetime.now() - start

In [ ]:
test_loss, test_accuracy = model.evaluate(test_cnn, test_labels)
print('test set accuracy: ', test_accuracy)
preds = model.predict(test_cnn)
print('shape of preds: ', preds.shape)
history_dict = history.history
history_dict.keys()
history_df=pd.DataFrame(history_dict)
#history_df.tail().round(3)
last_train_acc=history_df.tail(1).accuracy.values
last_val_acc=history_df.tail(1).val_accuracy.values
last_train_loss=history_df.tail(1).loss.values
last_val_loss=history_df.tail(1).val_loss.values

In [ ]:
losses = history.history['loss']
accs = history.history['accuracy']
val_losses = history.history['val_loss']
val_accs = history.history['val_accuracy']
epochs = len(losses)
plt.figure(figsize=(16, 4))
for i, metrics in enumerate(zip([losses, accs], [val_losses, val_accs], ['Loss', 'Accuracy'])):
    plt.subplot(1, 2, i + 1)
    plt.plot(range(epochs), metrics[0], label='Training {}'.format(metrics[2]))
    plt.plot(range(epochs), metrics[1], label='Validation {}'.format(metrics[2]))
    plt.legend()
plt.show()